In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
%cd /gdrive/My Drive/AN2DL/Homework1

/gdrive/My Drive/AN2DL/Homework1


### Import libraries

In [3]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
import splitfolders
tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

2.9.2


### Set seed for reproducibility

In [4]:
# Random seed for reproducibility
seed = 41

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

### Suppress warnings

In [5]:
import warnings
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

tf.get_logger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [6]:
# Download the data
dataset = '/gdrive/My Drive/AN2DL/Homework1/training_dataset_homework1'
training = '/gdrive/My Drive/AN2DL/Homework1/training_dataset_homework1/training'
validation = '/gdrive/My Drive/AN2DL/Homework1/training_dataset_homework1/validation'
test = '/gdrive/My Drive/AN2DL/Homework1/training_dataset_homework1/test'

In [7]:
labels = {
    0:'species1',
    1:'species2',
    2:'species3',
    3:'species4',
    4:'species5',
    5:'species6',
    6:'species7',
    7:'species8'
}

### Process data

In [14]:
# Images are divided into folders, one for each class.
# If the images are organized in such a way, we can exploit the
# ImageDataGenerator to read them from disk.
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import preprocess_input

# Create an instance of ImageDataGenerator for training, validation, and test sets
train_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input)
valid_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input)
test_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input)

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
train_gen = train_data_gen.flow_from_directory(directory=training,
                                               target_size=(96,96),
                                               color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=32,
                                               shuffle=True,
                                               seed=seed
                                               )
valid_gen = train_data_gen.flow_from_directory(directory=validation,
                                               target_size=(96,96),
                                               color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=32,
                                               shuffle=True,
                                               seed=seed
                                               )
test_gen = train_data_gen.flow_from_directory(directory=test,
                                              target_size=(96,96),
                                              color_mode='rgb',
                                              classes=None, # can be set to labels
                                              class_mode='categorical',
                                              batch_size=32,
                                              shuffle=False,
                                              seed=seed
                                              )

Found 2811 images belonging to 8 classes.
Found 336 images belonging to 8 classes.
Found 535 images belonging to 8 classes.


In [15]:
# Create an instance of ImageDataGenerator with Data Augmentation
aug_train_data_gen = ImageDataGenerator(rotation_range= 30,
                                        horizontal_flip= True,
                                        vertical_flip= True,
                                        brightness_range=(0.4,1.6),
                                        zoom_range=0.6,
                                        fill_mode='nearest',
                                        preprocessing_function = preprocess_input) # rescale value is multiplied to the image

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
aug_train_gen = aug_train_data_gen.flow_from_directory(directory=training,
                                                       target_size=(96,96),
                                                       color_mode='rgb',
                                                       classes=None, # can be set to labels
                                                       class_mode='categorical',
                                                       batch_size=32,
                                                       shuffle=True,
                                                       seed=seed)

Found 2811 images belonging to 8 classes.


### Models metadata

In [16]:
input_shape = (96, 96, 3)
batch_size = 32
epochs = 200

### Standard model

### Transfer learning

In [17]:
# Download and plot the VGG16 model
supernet = tfk.applications.DenseNet201(
    include_top=False,
    weights="imagenet",
    input_shape=(96,96,3)
)
supernet.summary()

74836368/74836368 [==============================] - 4s 0us/step
Model: "densenet201"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 96, 96, 3)]  0           []                               
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 102, 102, 3)  0          ['input_3[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 48, 48, 64)   9408        ['zero_padding2d[0][0]']         
                                                                                                  
 conv1/bn (BatchNormalization)  (None, 48, 48, 64)   256         ['conv1/conv[0][0]']             
                       

In [38]:
# Use the supernet as feature extractor
supernet.trainable = False

inputs = tfk.Input(shape=(96,96,3))
x = supernet(inputs)
x = tfkl.Flatten(name='Flattening')(x)
x = tfkl.Dropout(0.2, seed=seed)(x)
x = tfkl.Dense(
    1024, 
    activation='relu',
    kernel_initializer = tfk.initializers.HeUniform(seed))(x)
x = tfkl.Dense(
    256, 
    activation='relu',
    kernel_initializer = tfk.initializers.HeUniform(seed))(x)
x = tfkl.Dropout(0.2, seed=seed)(x)
x = tfkl.Dense(
    128, 
    activation='relu',
    kernel_initializer = tfk.initializers.HeUniform(seed))(x)
x = tfkl.Dropout(0.2, seed=seed)(x)
outputs = tfkl.Dense(
    8,
    activation='softmax',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)


# Connect input and output through the Model class
tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model
tl_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')
tl_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 densenet201 (Functional)    (None, 3, 3, 1920)        18321984  
                                                                 
 Flattening (Flatten)        (None, 17280)             0         
                                                                 
 dropout_6 (Dropout)         (None, 17280)             0         
                                                                 
 dense_7 (Dense)             (None, 1024)              17695744  
                                                                 
 dense_8 (Dense)             (None, 256)               262400    
                                                                 
 dropout_7 (Dropout)         (None, 256)               0     

In [39]:
# Train the model
tl_history = tl_model.fit(
    x = aug_train_gen,
    batch_size = 32,
    epochs = 200,
    validation_data = valid_gen,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)]
).history

Epoch 1/200
88/88 [==============================] - 26s 192ms/step - loss: 3.4610 - accuracy: 0.2540 - val_loss: 1.6039 - val_accuracy: 0.4048
Epoch 2/200
88/88 [==============================] - 14s 160ms/step - loss: 1.7657 - accuracy: 0.3266 - val_loss: 1.2677 - val_accuracy: 0.5327
Epoch 3/200
88/88 [==============================] - 14s 159ms/step - loss: 1.6280 - accuracy: 0.3767 - val_loss: 1.2493 - val_accuracy: 0.5417
Epoch 4/200
88/88 [==============================] - 14s 155ms/step - loss: 1.5691 - accuracy: 0.4077 - val_loss: 1.2906 - val_accuracy: 0.4881
Epoch 5/200
88/88 [==============================] - 14s 158ms/step - loss: 1.4983 - accuracy: 0.4354 - val_loss: 1.1566 - val_accuracy: 0.5595
Epoch 6/200
88/88 [==============================] - 14s 158ms/step - loss: 1.4414 - accuracy: 0.4593 - val_loss: 1.1917 - val_accuracy: 0.6042
Epoch 7/200
88/88 [==============================] - 14s 155ms/step - loss: 1.3998 - accuracy: 0.4646 - val_loss: 1.1822 - val_accuracy:

In [40]:
# Save the best model

tl_model.save('TransferLearningModel')
del tl_model

In [41]:
model_aug = tfk.models.load_model("TransferLearningModel")
model_aug_test_metrics = model_aug.evaluate(test_gen, return_dict=True)

print()
print("Test metrics with VGG")
print(model_aug_test_metrics)

17/17 [==============================] - 4s 70ms/step - loss: 1.1130 - accuracy: 0.5925

Test metrics with VGG
{'loss': 1.1130439043045044, 'accuracy': 0.5925233364105225}


### Fine tuning

In [42]:
# Re-load the model after transfer learning
ft_model = tfk.models.load_model('TransferLearningModel')
ft_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 densenet201 (Functional)    (None, 3, 3, 1920)        18321984  
                                                                 
 Flattening (Flatten)        (None, 17280)             0         
                                                                 
 dropout_6 (Dropout)         (None, 17280)             0         
                                                                 
 dense_7 (Dense)             (None, 1024)              17695744  
                                                                 
 dense_8 (Dense)             (None, 256)               262400    
                                                                 
 dropout_7 (Dropout)         (None, 256)               0     

In [43]:
# Set all VGG layers to True
ft_model.get_layer('densenet201').trainable = True
for i, layer in enumerate(ft_model.get_layer('densenet201').layers):
   print(i, layer.name, layer.trainable)

0 input_3 True
1 zero_padding2d True
2 conv1/conv True
3 conv1/bn True
4 conv1/relu True
5 zero_padding2d_1 True
6 pool1 True
7 conv2_block1_0_bn True
8 conv2_block1_0_relu True
9 conv2_block1_1_conv True
10 conv2_block1_1_bn True
11 conv2_block1_1_relu True
12 conv2_block1_2_conv True
13 conv2_block1_concat True
14 conv2_block2_0_bn True
15 conv2_block2_0_relu True
16 conv2_block2_1_conv True
17 conv2_block2_1_bn True
18 conv2_block2_1_relu True
19 conv2_block2_2_conv True
20 conv2_block2_concat True
21 conv2_block3_0_bn True
22 conv2_block3_0_relu True
23 conv2_block3_1_conv True
24 conv2_block3_1_bn True
25 conv2_block3_1_relu True
26 conv2_block3_2_conv True
27 conv2_block3_concat True
28 conv2_block4_0_bn True
29 conv2_block4_0_relu True
30 conv2_block4_1_conv True
31 conv2_block4_1_bn True
32 conv2_block4_1_relu True
33 conv2_block4_2_conv True
34 conv2_block4_concat True
35 conv2_block5_0_bn True
36 conv2_block5_0_relu True
37 conv2_block5_1_conv True
38 conv2_block5_1_bn True
3

In [44]:
# Freeze first N layers, e.g., until 14th
for i, layer in enumerate(ft_model.get_layer('densenet201').layers[:460]):
  layer.trainable=False
for i, layer in enumerate(ft_model.get_layer('densenet201').layers):
   print(i, layer.name, layer.trainable)
ft_model.summary()

0 input_3 False
1 zero_padding2d False
2 conv1/conv False
3 conv1/bn False
4 conv1/relu False
5 zero_padding2d_1 False
6 pool1 False
7 conv2_block1_0_bn False
8 conv2_block1_0_relu False
9 conv2_block1_1_conv False
10 conv2_block1_1_bn False
11 conv2_block1_1_relu False
12 conv2_block1_2_conv False
13 conv2_block1_concat False
14 conv2_block2_0_bn False
15 conv2_block2_0_relu False
16 conv2_block2_1_conv False
17 conv2_block2_1_bn False
18 conv2_block2_1_relu False
19 conv2_block2_2_conv False
20 conv2_block2_concat False
21 conv2_block3_0_bn False
22 conv2_block3_0_relu False
23 conv2_block3_1_conv False
24 conv2_block3_1_bn False
25 conv2_block3_1_relu False
26 conv2_block3_2_conv False
27 conv2_block3_concat False
28 conv2_block4_0_bn False
29 conv2_block4_0_relu False
30 conv2_block4_1_conv False
31 conv2_block4_1_bn False
32 conv2_block4_1_relu False
33 conv2_block4_2_conv False
34 conv2_block4_concat False
35 conv2_block5_0_bn False
36 conv2_block5_0_relu False
37 conv2_block5_1_

In [45]:
# Compile the model
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-4), metrics='accuracy')

In [46]:
# Fine-tune the model
ft_history = ft_model.fit(
    x = aug_train_gen,
    batch_size = 32,
    epochs = 200,
    validation_data = valid_gen,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)]
).history

Epoch 1/200
88/88 [==============================] - 31s 209ms/step - loss: 1.2260 - accuracy: 0.5607 - val_loss: 0.9330 - val_accuracy: 0.6518
Epoch 2/200
88/88 [==============================] - 15s 170ms/step - loss: 1.0364 - accuracy: 0.6290 - val_loss: 0.9293 - val_accuracy: 0.6905
Epoch 3/200
88/88 [==============================] - 15s 170ms/step - loss: 1.0004 - accuracy: 0.6471 - val_loss: 0.8737 - val_accuracy: 0.7054
Epoch 4/200
88/88 [==============================] - 15s 170ms/step - loss: 0.9569 - accuracy: 0.6539 - val_loss: 0.7447 - val_accuracy: 0.7649
Epoch 5/200
88/88 [==============================] - 15s 167ms/step - loss: 0.8779 - accuracy: 0.6941 - val_loss: 0.7953 - val_accuracy: 0.7262
Epoch 6/200
88/88 [==============================] - 15s 165ms/step - loss: 0.8684 - accuracy: 0.6969 - val_loss: 0.7177 - val_accuracy: 0.7500
Epoch 7/200
88/88 [==============================] - 15s 166ms/step - loss: 0.7831 - accuracy: 0.7275 - val_loss: 0.7577 - val_accuracy:

In [47]:
ft_model.save('FineTuningModel')
del ft_model

In [48]:
model_aug = tfk.models.load_model("FineTuningModel")
model_aug_test_metrics = model_aug.evaluate(test_gen, return_dict=True)

print()
print("Test metrics with VGG")
print(model_aug_test_metrics)

17/17 [==============================] - 4s 70ms/step - loss: 0.4770 - accuracy: 0.8561

Test metrics with VGG
{'loss': 0.47698211669921875, 'accuracy': 0.8560747504234314}
